In [2]:
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from keras.layers import Dense, Input, concatenate, GRU, LSTM
from keras import backend as K
from keras.utils import to_categorical
from keras import Model
from nltk.translate.bleu_score import sentence_bleu

Using TensorFlow backend.


In [3]:
iris = load_iris()
X = iris['data']
y = iris['target']
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [4]:
df = pd.DataFrame(X)

In [5]:
bin_labels = pd.read_csv('../../data/raw/rpart_bin_labels.csv', delimiter=",")

In [6]:
bin_labels = bin_labels.rename(columns={"Unnamed: 0": "label", "label_list": "bins"})

In [7]:
# path_df = pd.read_csv('../../data/raw/rpart_paths.csv', delimiter=",")
# path_df = pd.read_csv('../../data/raw/test_paths.csv', delimiter=",")
path_df = pd.read_csv('../../data/raw/paths.csv', delimiter=",")

In [8]:
path_df = path_df.drop(["Unnamed: 0"], axis=1)
path_df = path_df.rename(columns={list(path_df)[0]: "new_col"})

In [9]:
test_data = pd.concat([df, path_df], axis=1)
# test_data = df

In [10]:
bin_labels

,label,bins
0,A,0.80
1,B,1.35
2,C,1.55
3,D,1.65
4,E,1.75
5,F,1.85
6,G,2.45
7,H,2.55
8,I,2.65
9,J,2.85


In [11]:
test_data

,0,1,2,3,new_col
0,-0.900681,1.019004,-1.340227,-1.315444,3C0
1,-1.143017,-0.131979,-1.340227,-1.315444,1K0
2,-1.385353,0.328414,-1.397064,-1.315444,3O0
3,-1.506521,0.098217,-1.283389,-1.315444,"3Y0,1AB0"
4,-1.021849,1.249201,-1.340227,-1.315444,3AF0
5,-0.537178,1.939791,-1.169714,-1.052180,1AR0
6,-1.506521,0.788808,-1.340227,-1.183812,"3AZ0,1BD0"
7,-1.021849,0.788808,-1.283389,-1.315444,3BG0
8,-1.748856,-0.362176,-1.340227,-1.315444,"3BQ0,1BR0"
9,-1.143017,0.098217,-1.283389,-1.447076,1BZ0


In [12]:
new_path = []
for i, val in test_data.iterrows():
    new_path.append(val['new_col'].split(sep=","))

In [13]:
test_data.head()

,0,1,2,3,new_col
0,-0.900681,1.019004,-1.340227,-1.315444,3C0
1,-1.143017,-0.131979,-1.340227,-1.315444,1K0
2,-1.385353,0.328414,-1.397064,-1.315444,3O0
3,-1.506521,0.098217,-1.283389,-1.315444,"3Y0,1AB0"
4,-1.021849,1.249201,-1.340227,-1.315444,3AF0


In [14]:
[x.insert(0, 'S') for x in new_path]
[x.append('E') for x in new_path]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [15]:
new_path

[['S', '3C0', 'E'],
 ['S', '1K0', 'E'],
 ['S', '3O0', 'E'],
 ['S', '3Y0', '1AB0', 'E'],
 ['S', '3AF0', 'E'],
 ['S', '1AR0', 'E'],
 ['S', '3AZ0', '1BD0', 'E'],
 ['S', '3BG0', 'E'],
 ['S', '3BQ0', '1BR0', 'E'],
 ['S', '1BZ0', 'E'],
 ['S', '3CG0', 'E'],
 ['S', '1CS0', 'E'],
 ['S', '3CY0', '1CZ0', 'E'],
 ['S', '1DN0', 'E'],
 ['S', '3DR0', 'E'],
 ['S', '2EG1', 'E'],
 ['S', '4EL0', '1EP0', 'E'],
 ['S', '2EX1', 'E'],
 ['S', '3FQ0', '2FP1', 'E'],
 ['S', '1FZ0', 'E'],
 ['S', '1GP0', '2GL1', 'E'],
 ['S', '1GX0', '1GW0', 'E'],
 ['S', '1HM0', 'E'],
 ['S', '1HU0', 'E'],
 ['S', '1IM0', 'E'],
 ['S', '4IQ0', '1IW0', 'E'],
 ['S', '1JQ0', '2JN1', 'E'],
 ['S', '1JX0', 'E'],
 ['S', '1KQ0', 'E'],
 ['S', '3KY0', '1KY0', 'E'],
 ['S', '1LT0', 'E'],
 ['S', '2MN1', 'E'],
 ['S', '1NO0', 'E'],
 ['S', '1NW0', 'E'],
 ['S', '1PQ0', '1OY0', 'E'],
 ['S', '2PU1', '1PY0', 'E'],
 ['S', '1QX0', '3QS0', 'E'],
 ['S', '3RV0', '1RX0', 'E'],
 ['S', '3SX0', '1SW0', 'E'],
 ['S', '1UV0', 'E'],
 ['S', '1WY0', 'E'],
 ['S', '1ABG0',

In [16]:
test_data['new_path'] = new_path

In [16]:
3G1', '4E0', '3R0', '4D0
3G1', '4E1', '3Q1
a = [1,0,0,0]
b = [1,1,1]
c = [0]

SyntaxError: invalid syntax (<ipython-input-16-103b982836d5>, line 1)

In [17]:
test_data = test_data.drop(["new_col"], axis=1)

In [18]:
test_data.head()

,0,1,2,3,new_path
0,-0.900681,1.019004,-1.340227,-1.315444,"[S, 3C0, E]"
1,-1.143017,-0.131979,-1.340227,-1.315444,"[S, 1K0, E]"
2,-1.385353,0.328414,-1.397064,-1.315444,"[S, 3O0, E]"
3,-1.506521,0.098217,-1.283389,-1.315444,"[S, 3Y0, 1AB0, E]"
4,-1.021849,1.249201,-1.340227,-1.315444,"[S, 3AF0, E]"


In [19]:
paths_lengths = np.array([len(xi)
                          for xi in test_data.iloc[:,-1]])

In [20]:
paths_lengths
np.max(paths_lengths)

5

In [21]:
label_char = []
for _, i in enumerate(np.unique(test_data['new_path'])):
    for _, j in enumerate(i):
        if j not in label_char:
            label_char.append(j)

In [22]:
label_indices = { j : i for i, j in enumerate(label_char) }

In [23]:
label_indices

{'S': 0,
 '1ABG0': 1,
 'E': 2,
 '1ACD0': 3,
 '1ADF0': 4,
 '1ADN0': 5,
 '1ADW0': 6,
 '1AER1': 7,
 '3AEQ0': 8,
 '1AFN1': 9,
 '2AFJ1': 10,
 '1AFV1': 11,
 '3AFU0': 12,
 '1AHN1': 13,
 '3AHM0': 14,
 '1AHW0': 15,
 '2AHT0': 16,
 '1AIM1': 17,
 '3AIL0': 18,
 '1AJV1': 19,
 '3AJU0': 20,
 '1AKW0': 21,
 '2AKT0': 22,
 '1ALP1': 23,
 '3ALO0': 24,
 '1ALX1': 25,
 '3ALW0': 26,
 '1AMS1': 27,
 '4AMN0': 28,
 '1ANV1': 29,
 '3ANU0': 30,
 '1AOS1': 31,
 '1AOT0': 32,
 '1AOZ1': 33,
 '1APR0': 34,
 '1AR0': 35,
 '1ARW1': 36,
 '1ARX0': 37,
 '1AVX1': 38,
 '3AVW0': 39,
 '1BCE1': 40,
 '4AWY0': 41,
 '1BCP0': 42,
 '2BCM0': 43,
 '1BDH1': 44,
 '1BDI0': 45,
 '1BFQ1': 46,
 '4BFL0': 47,
 '1BGP0': 48,
 '1BGO1': 49,
 '1BHM1': 50,
 '4BGZ0': 51,
 '1BIL1': 52,
 '4BHX0': 53,
 '1BKP1': 54,
 '3BKO0': 55,
 '1BKX1': 56,
 '4BKS0': 57,
 '1BLZ1': 58,
 '1BMN0': 59,
 '1BZ0': 60,
 '1CDF1': 61,
 '3CDE1': 62,
 '1CHJ1': 63,
 '1CHK0': 64,
 '1CHR1': 65,
 '3CHQ1': 66,
 '1CJO1': 67,
 '1CKV1': 68,
 '1CLO1': 69,
 '1CLP1': 70,
 '1CLX1': 71,
 '1CNP1': 72

In [25]:
len(label_indices)

240

In [58]:
indices_label = { i : j for i, j in enumerate(label_char) }
indices_label

{0: 'S',
 1: '1ABG0',
 2: 'E',
 3: '1ACD0',
 4: '1ADF0',
 5: '1ADN0',
 6: '1ADW0',
 7: '1AER1',
 8: '3AEQ0',
 9: '1AFN1',
 10: '2AFJ1',
 11: '1AFV1',
 12: '3AFU0',
 13: '1AHN1',
 14: '3AHM0',
 15: '1AHW0',
 16: '2AHT0',
 17: '1AIM1',
 18: '3AIL0',
 19: '1AJV1',
 20: '3AJU0',
 21: '1AKW0',
 22: '2AKT0',
 23: '1ALP1',
 24: '3ALO0',
 25: '1ALX1',
 26: '3ALW0',
 27: '1AMS1',
 28: '4AMN0',
 29: '1ANV1',
 30: '3ANU0',
 31: '1AOS1',
 32: '1AOT0',
 33: '1AOZ1',
 34: '1APR0',
 35: '1AR0',
 36: '1ARW1',
 37: '1ARX0',
 38: '1AVX1',
 39: '3AVW0',
 40: '1BCE1',
 41: '4AWY0',
 42: '1BCP0',
 43: '2BCM0',
 44: '1BDH1',
 45: '1BDI0',
 46: '1BFQ1',
 47: '4BFL0',
 48: '1BGP0',
 49: '1BGO1',
 50: '1BHM1',
 51: '4BGZ0',
 52: '1BIL1',
 53: '4BHX0',
 54: '1BKP1',
 55: '3BKO0',
 56: '1BKX1',
 57: '4BKS0',
 58: '1BLZ1',
 59: '1BMN0',
 60: '1BZ0',
 61: '1CDF1',
 62: '3CDE1',
 63: '1CHJ1',
 64: '1CHK0',
 65: '1CHR1',
 66: '3CHQ1',
 67: '1CJO1',
 68: '1CKV1',
 69: '1CLO1',
 70: '1CLP1',
 71: '1CLX1',
 72: '1CNP1'

In [24]:
bin_labels

,label,bins
0,A,0.80
1,B,1.35
2,C,1.55
3,D,1.65
4,E,1.75
5,F,1.85
6,G,2.45
7,H,2.55
8,I,2.65
9,J,2.85


In [28]:
np.unique(path_df, return_counts=True)

(array(['1ABG0', '1ACD0', '1ADF0', '1ADN0', '1ADW0', '1AER1,3AEQ0',
        '1AFN1,2AFJ1', '1AFV1,3AFU0', '1AHN1,3AHM0', '1AHW0,2AHT0',
        '1AIM1,3AIL0', '1AJV1,3AJU0', '1AKW0,2AKT0', '1ALP1,3ALO0',
        '1ALX1,3ALW0', '1AMS1,4AMN0', '1ANV1,3ANU0', '1AOS1,1AOT0',
        '1AOZ1,1APR0', '1AR0', '1ARW1,1ARX0', '1AVX1,3AVW0', '1BCE1,4AWY0',
        '1BCP0,2BCM0', '1BDH1,1BDI0', '1BFQ1,4BFL0', '1BGP0,1BGO1',
        '1BHM1,4BGZ0', '1BIL1,4BHX0', '1BKP1,3BKO0', '1BKX1,4BKS0',
        '1BLZ1,1BMN0', '1BZ0', '1CDF1,3CDE1', '1CHJ1,1CHK0', '1CHR1,3CHQ1',
        '1CJO1', '1CKV1', '1CLO1,1CLP1', '1CLX1', '1CNP1,3CMZ1,2CMX0',
        '1COX1,3COW1', '1CPW1', '1CQU1,3CQT1', '1CRX1,1CRZ1', '1CS0',
        '1DEH1,1DEI1', '1DEZ1', '1DGI1,3DGH1', '1DN0', '1FZ0', '1GP0,2GL1',
        '1GX0,1GW0', '1HM0', '1HU0', '1IM0', '1JQ0,2JN1', '1JX0', '1K0',
        '1KQ0', '1LT0', '1NO0', '1NW0', '1PQ0,1OY0', '1QX0,3QS0', '1UV0',
        '1WY0', '2ACJ1', '2APW0,4APT0', '2BDM0', '2CEJ0,1CEO0,1CEN1',
      

In [29]:
path_df

,new_col
0,3C0
1,1K0
2,3O0
3,"3Y0,1AB0"
4,3AF0
5,1AR0
6,"3AZ0,1BD0"
7,3BG0
8,"3BQ0,1BR0"
9,1BZ0


In [59]:
input_path_sequence = []
next_chars = []
features = []
paths_maxlen = np.max(paths_lengths)
# path_vocab_size = len(bin_labels) # How is this working? Validate!
path_vocab_size = len(indices_label) # Temporary test for local trees
feature_size = 4
for i in range(0, len(test_data)):
    # get the feature
    curr_feat = np.array([test_data.iloc[i, 0:4]])
    curr_path = test_data.iloc[i, -1]
    curr_path_len = len(curr_path)
    # curr_label = y[i]
    # curr_dec_feat = df.iloc[i, 6]
    for j in range(1, curr_path_len):
        features.append(curr_feat)
        input_path_sequence.append(curr_path[0:j])
        next_chars.append(curr_path[j])

x_path = np.zeros(
    (len(input_path_sequence), paths_maxlen, path_vocab_size), dtype=np.bool)

path_latent_input = np.zeros(
    (len(input_path_sequence), feature_size), dtype=np.float)

y_path = np.zeros(
    (len(input_path_sequence), path_vocab_size), dtype=np.bool)

# print(input_path_sequence)
# print(len(input_path_sequence))
for i, sentence in enumerate(input_path_sequence):
    for t, char in enumerate(sentence):
        # x_path[i, t, self.char_indices[char]] = 1
        # print(bin_labels.index[bin_labels['label'] == char[1]])
        # index = bin_labels.index[bin_labels['label'] == char[1]].tolist()[0]
        x_path[i, t, label_indices[char]] = 1
    # y_path[i, char_indices[next_chars[i]]] = 1
    # index = bin_labels.index[bin_labels['label'] == next_chars[i][1]].tolist()[0]
    # y_path[i, index] = 1
    y_path[i, label_indices[next_chars[i]]] = 1
    path_latent_input[i, :] = features[i]
    
## Trouble with "S" and "E" index values.

In [61]:
len(input_path_sequence)

388

In [63]:
len(next_chars)

388

In [64]:
y_path.shape

(388, 240)

In [65]:
def _create_label_model(latent_dim=5):
    input_layer = Input(shape=(feature_size,), name='ip_x')
    hidden_layer_x1 = Dense(20, activation='tanh',
                            name='hidden_x1')(input_layer)
    hidden_layer_x2 = Dense(20, activation='tanh',
                            name='hidden_x2')(hidden_layer_x1)
    hidden_layer_x3 = Dense(latent_dim, activation='tanh',
                            name='hidden_x3')(hidden_layer_x2)
    output_layer = Dense(len(np.unique(y)), activation='softmax',
                         name='op_x')(hidden_layer_x3)
    model = Model(input_layer, output_layer)
    return model

def _create_combined_model(initialize=True, rnn_cell='gru', latent_dim=5):

    label_model_latent = Input(shape=(latent_dim,), name='label_ip')
    path_input = Input(shape=(
        paths_maxlen, path_vocab_size), name='dec_feat_ip')
    if rnn_cell == 'gru':
        RNN = GRU
    else:
        RNN = LSTM

    decoder = RNN(latent_dim, return_state=False,
                  return_sequences=False, name='gru_seq')
    if initialize:
        decoder_outputs = decoder(
            path_input, initial_state=label_model_latent)
    else:
        decoder_outputs = decoder(path_input)

    merge_layer = concatenate(
        [label_model_latent, decoder_outputs], name='cat')
    output_chars = Dense(path_vocab_size,
                         activation='softmax', name='op_sent')(merge_layer)
    model = Model(
        [label_model_latent, path_input], output_chars)
    return model

In [66]:
combined_model = _create_combined_model()
label_model = _create_label_model()

In [67]:
def get_hidden_x(x, model, layer_num=3):
    def get_hidden_x_inner(model, layer_num=layer_num):
        return K.function([model.layers[0].input], [model.layers[layer_num].output])
    return get_hidden_x_inner(model, layer_num=layer_num)([x])[0]

In [79]:
def fit_model():

    y_cat = to_categorical(y)

    label_model.compile(
        optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    label_model.fit(
        X, y_cat, batch_size=30, epochs=150, verbose=0, shuffle=True, validation_split=0.2)

    x_latent = get_hidden_x(path_latent_input, model=label_model)

    combined_model.compile(
        optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    combined_model.fit([x_latent, x_path], y_path,
                           batch_size=30, epochs=5000, verbose=1, shuffle=True)

In [80]:
fit_model()

Epoch 1/5000
388/388 [==============================] - 1s 2ms/step - loss: 2.6952 - acc: 0.3943
Epoch 2/5000
388/388 [==============================] - 0s 145us/step - loss: 2.6826 - acc: 0.3943
Epoch 3/5000
388/388 [==============================] - 0s 123us/step - loss: 2.6806 - acc: 0.3969
Epoch 4/5000
388/388 [==============================] - 0s 135us/step - loss: 2.6796 - acc: 0.3969
Epoch 5/5000
388/388 [==============================] - 0s 121us/step - loss: 2.6789 - acc: 0.3969
Epoch 6/5000
388/388 [==============================] - 0s 131us/step - loss: 2.6777 - acc: 0.3943
Epoch 7/5000
388/388 [==============================] - 0s 131us/step - loss: 2.6766 - acc: 0.3918
Epoch 8/5000
388/388 [==============================] - 0s 150us/step - loss: 2.6759 - acc: 0.3969
Epoch 9/5000
388/388 [==============================] - 0s 119us/step - loss: 2.6745 - acc: 0.3918
Epoch 10/5000
388/388 [==============================] - 0s 109us/step - loss: 2.6736 - acc: 0.4021
Epoch 11/50

388/388 [==============================] - 0s 118us/step - loss: 2.4368 - acc: 0.4072
Epoch 84/5000
388/388 [==============================] - 0s 108us/step - loss: 2.4356 - acc: 0.4072
Epoch 85/5000
388/388 [==============================] - 0s 99us/step - loss: 2.4344 - acc: 0.4072
Epoch 86/5000
388/388 [==============================] - 0s 109us/step - loss: 2.4333 - acc: 0.4046
Epoch 87/5000
388/388 [==============================] - 0s 94us/step - loss: 2.4321 - acc: 0.4072
Epoch 88/5000
388/388 [==============================] - 0s 110us/step - loss: 2.4310 - acc: 0.4124
Epoch 89/5000
388/388 [==============================] - 0s 96us/step - loss: 2.4298 - acc: 0.4124
Epoch 90/5000
388/388 [==============================] - 0s 99us/step - loss: 2.4287 - acc: 0.4072
Epoch 91/5000
388/388 [==============================] - 0s 102us/step - loss: 2.4280 - acc: 0.4046
Epoch 92/5000
388/388 [==============================] - 0s 113us/step - loss: 2.4269 - acc: 0.4072
Epoch 93/5000
388/

388/388 [==============================] - 0s 126us/step - loss: 2.2870 - acc: 0.4278
Epoch 246/5000
388/388 [==============================] - 0s 122us/step - loss: 2.2862 - acc: 0.4304
Epoch 247/5000
388/388 [==============================] - 0s 154us/step - loss: 2.2856 - acc: 0.4278
Epoch 248/5000
388/388 [==============================] - 0s 171us/step - loss: 2.2850 - acc: 0.4253
Epoch 249/5000
388/388 [==============================] - 0s 134us/step - loss: 2.2843 - acc: 0.4304
Epoch 250/5000
388/388 [==============================] - 0s 128us/step - loss: 2.2839 - acc: 0.4304
Epoch 251/5000
388/388 [==============================] - 0s 152us/step - loss: 2.2832 - acc: 0.4304
Epoch 252/5000
388/388 [==============================] - 0s 149us/step - loss: 2.2827 - acc: 0.4253
Epoch 253/5000
388/388 [==============================] - 0s 128us/step - loss: 2.2819 - acc: 0.4227
Epoch 254/5000
388/388 [==============================] - 0s 141us/step - loss: 2.2814 - acc: 0.4253
Epoch

388/388 [==============================] - 0s 107us/step - loss: 2.2045 - acc: 0.4304
Epoch 408/5000
388/388 [==============================] - 0s 117us/step - loss: 2.2033 - acc: 0.4381
Epoch 409/5000
388/388 [==============================] - 0s 113us/step - loss: 2.2026 - acc: 0.4356
Epoch 410/5000
388/388 [==============================] - 0s 111us/step - loss: 2.2025 - acc: 0.4356
Epoch 411/5000
388/388 [==============================] - 0s 114us/step - loss: 2.2018 - acc: 0.4356
Epoch 412/5000
388/388 [==============================] - 0s 118us/step - loss: 2.2014 - acc: 0.4253
Epoch 413/5000
388/388 [==============================] - 0s 104us/step - loss: 2.2015 - acc: 0.4356
Epoch 414/5000
388/388 [==============================] - 0s 109us/step - loss: 2.2005 - acc: 0.4381
Epoch 415/5000
388/388 [==============================] - 0s 111us/step - loss: 2.2006 - acc: 0.4330
Epoch 416/5000
388/388 [==============================] - 0s 113us/step - loss: 2.1998 - acc: 0.4278
Epoch

388/388 [==============================] - 0s 117us/step - loss: 2.1395 - acc: 0.4459
Epoch 570/5000
388/388 [==============================] - 0s 120us/step - loss: 2.1391 - acc: 0.4407
Epoch 571/5000
388/388 [==============================] - 0s 120us/step - loss: 2.1387 - acc: 0.4407
Epoch 572/5000
388/388 [==============================] - 0s 116us/step - loss: 2.1383 - acc: 0.4407
Epoch 573/5000
388/388 [==============================] - 0s 135us/step - loss: 2.1380 - acc: 0.4485
Epoch 574/5000
388/388 [==============================] - 0s 118us/step - loss: 2.1377 - acc: 0.4381
Epoch 575/5000
388/388 [==============================] - 0s 117us/step - loss: 2.1375 - acc: 0.4381
Epoch 576/5000
388/388 [==============================] - 0s 110us/step - loss: 2.1371 - acc: 0.4459
Epoch 577/5000
388/388 [==============================] - 0s 106us/step - loss: 2.1368 - acc: 0.4459
Epoch 578/5000
388/388 [==============================] - 0s 117us/step - loss: 2.1367 - acc: 0.4356
Epoch

...

388/388 [==============================] - 0s 155us/step - loss: 1.8710 - acc: 0.5103
Epoch 4335/5000
388/388 [==============================] - 0s 126us/step - loss: 1.8757 - acc: 0.5232
Epoch 4336/5000
388/388 [==============================] - 0s 113us/step - loss: 1.8702 - acc: 0.5129
Epoch 4337/5000
388/388 [==============================] - 0s 107us/step - loss: 1.8803 - acc: 0.5052
Epoch 4338/5000
388/388 [==============================] - 0s 120us/step - loss: 1.8804 - acc: 0.5129
Epoch 4339/5000
388/388 [==============================] - 0s 144us/step - loss: 1.8789 - acc: 0.5155
Epoch 4340/5000
388/388 [==============================] - 0s 137us/step - loss: 1.8714 - acc: 0.5052
Epoch 4341/5000
388/388 [==============================] - 0s 142us/step - loss: 1.8734 - acc: 0.5026
Epoch 4342/5000
388/388 [==============================] - 0s 123us/step - loss: 1.8812 - acc: 0.5026
Epoch 4343/5000
388/388 [==============================] - 0s 120us/step - loss: 1.8818 - acc: 0.5

388/388 [==============================] - 0s 127us/step - loss: 1.8785 - acc: 0.5155
Epoch 4495/5000
388/388 [==============================] - 0s 130us/step - loss: 1.8726 - acc: 0.5052
Epoch 4496/5000
388/388 [==============================] - 0s 124us/step - loss: 1.8774 - acc: 0.5052
Epoch 4497/5000
388/388 [==============================] - 0s 113us/step - loss: 1.8734 - acc: 0.4923
Epoch 4498/5000
388/388 [==============================] - 0s 114us/step - loss: 1.8763 - acc: 0.5077
Epoch 4499/5000
388/388 [==============================] - 0s 164us/step - loss: 1.8736 - acc: 0.5026
Epoch 4500/5000
388/388 [==============================] - 0s 114us/step - loss: 1.8712 - acc: 0.5232
Epoch 4501/5000
388/388 [==============================] - 0s 132us/step - loss: 1.8779 - acc: 0.5026
Epoch 4502/5000
388/388 [==============================] - 0s 111us/step - loss: 1.8776 - acc: 0.4974
Epoch 4503/5000
388/388 [==============================] - 0s 114us/step - loss: 1.8765 - acc: 0.4

388/388 [==============================] - 0s 118us/step - loss: 1.8721 - acc: 0.5026
Epoch 4655/5000
388/388 [==============================] - 0s 123us/step - loss: 1.8727 - acc: 0.5129
Epoch 4656/5000
388/388 [==============================] - 0s 128us/step - loss: 1.8725 - acc: 0.4948
Epoch 4657/5000
388/388 [==============================] - 0s 128us/step - loss: 1.8728 - acc: 0.5206
Epoch 4658/5000
388/388 [==============================] - 0s 104us/step - loss: 1.8697 - acc: 0.5180
Epoch 4659/5000
388/388 [==============================] - 0s 119us/step - loss: 1.8663 - acc: 0.5103
Epoch 4660/5000
388/388 [==============================] - 0s 119us/step - loss: 1.8737 - acc: 0.5232
Epoch 4661/5000
388/388 [==============================] - 0s 122us/step - loss: 1.8764 - acc: 0.5026
Epoch 4662/5000
388/388 [==============================] - 0s 118us/step - loss: 1.8760 - acc: 0.5000
Epoch 4663/5000
388/388 [==============================] - 0s 107us/step - loss: 1.8726 - acc: 0.5

388/388 [==============================] - 0s 120us/step - loss: 1.8713 - acc: 0.5129
Epoch 4815/5000
388/388 [==============================] - 0s 113us/step - loss: 1.8683 - acc: 0.5000
Epoch 4816/5000
388/388 [==============================] - 0s 120us/step - loss: 1.8637 - acc: 0.5077
Epoch 4817/5000
388/388 [==============================] - 0s 113us/step - loss: 1.8643 - acc: 0.5000
Epoch 4818/5000
388/388 [==============================] - 0s 113us/step - loss: 1.8690 - acc: 0.5077
Epoch 4819/5000
388/388 [==============================] - 0s 120us/step - loss: 1.8695 - acc: 0.5077
Epoch 4820/5000
388/388 [==============================] - 0s 107us/step - loss: 1.8680 - acc: 0.5026
Epoch 4821/5000
388/388 [==============================] - 0s 113us/step - loss: 1.8676 - acc: 0.5103
Epoch 4822/5000
388/388 [==============================] - 0s 116us/step - loss: 1.8602 - acc: 0.5180
Epoch 4823/5000
388/388 [==============================] - 0s 113us/step - loss: 1.8624 - acc: 0.5

388/388 [==============================] - 0s 123us/step - loss: 1.8555 - acc: 0.5077
Epoch 4975/5000
388/388 [==============================] - 0s 141us/step - loss: 1.8673 - acc: 0.5077
Epoch 4976/5000
388/388 [==============================] - 0s 132us/step - loss: 1.8658 - acc: 0.5103
Epoch 4977/5000
388/388 [==============================] - 0s 125us/step - loss: 1.8649 - acc: 0.5206
Epoch 4978/5000
388/388 [==============================] - 0s 135us/step - loss: 1.8652 - acc: 0.5129
Epoch 4979/5000
388/388 [==============================] - 0s 131us/step - loss: 1.8647 - acc: 0.5232
Epoch 4980/5000
388/388 [==============================] - 0s 121us/step - loss: 1.8645 - acc: 0.5129
Epoch 4981/5000
388/388 [==============================] - 0s 150us/step - loss: 1.8648 - acc: 0.5103
Epoch 4982/5000
388/388 [==============================] - 0s 136us/step - loss: 1.8639 - acc: 0.5284
Epoch 4983/5000
388/388 [==============================] - 0s 127us/step - loss: 1.8637 - acc: 0.5

In [71]:
combined_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
label_ip (InputLayer)           (None, 5)            0                                            
__________________________________________________________________________________________________
dec_feat_ip (InputLayer)        (None, 5, 240)       0                                            
__________________________________________________________________________________________________
gru_seq (GRU)                   (None, 5)            3690        dec_feat_ip[0][0]                
                                                                 label_ip[0][0]                   
__________________________________________________________________________________________________
cat (Concatenate)               (None, 10)           0           label_ip[0][0]                   
          

In [30]:
y_path

NameError: name 'y_path' is not defined

In [72]:
def predict(x):
    latent_dim = 5
    x_f = x.reshape(1, feature_size)
    token = 'S'
    cont = True
    path = [token]
    x_path = np.zeros((1, paths_maxlen, path_vocab_size), dtype=np.bool)

    x_latent = get_hidden_x(x_f, model=label_model)
    x_latent = x_latent.reshape(1, latent_dim)
    x_path[0, 0, label_indices[token]] = 1
    pred = label_model.predict(x_f)
    label = [np.argmax(pred[0])]
    index = 1
    while cont & (index < paths_maxlen):
        pred = combined_model.predict([x_latent, x_path])
        char_index = np.argmax(pred[0])
        x_path[0, index, char_index] = 1
        next_char = indices_label[char_index]
        path.append(next_char)
        index += 1
        if next_char == 'E':
            cont = False
        # elif index == self.paths_maxlen - 1:
        #     path.append('E')

    if path[-1] != 'E':
        path.append('E')

    return [path, label]

In [73]:
def check_path(path): # Returns -1 if path traversed is wrong/non-existant
    # path = ''.join(path)
    path = path[1:-1]
    pred_features = []
    path_as_strings = []
    for i in range(len(path)):
        pred_features.append(int(path[i][:-1]))
        path_as_strings.append(path[i][-1])
        # if i%2 == 0:
        #     print('i -- ', i)
        #     print('path -- ', path)
        #     print('path[i] -- ', path[i])
        #     pred_features.append(int(path[i]))
        # else:
        #     path_as_strings.append(path[i])

    n_nodes = self.clf.tree_.node_count
    children_left = self.clf.tree_.children_left
    children_right = self.clf.tree_.children_right
    feature = self.clf.tree_.feature

    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        # node_depth[node_id] = parent_depth + 1

        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            is_leaves[node_id] = True


    node = 0
    pred_target = -1
    subset_path = False
    for i in range(len(path_as_strings)):
        if path_as_strings[i] == 'L':
            if feature[node]+1 == pred_features[i]:
                node = children_left[node]
            # else:
                # pred_target = -1 # Remove for "subset" checks
                # break
        elif path_as_strings[i] == 'R':
            if feature[node]+1 == pred_features[i]:
                node = children_right[node]
            # else:
                # pred_target = -1 # Remove for "subset" checks
                # break
        if is_leaves[node]:
            for i, x in enumerate(self.clf.tree_.value[node][0]):
                if x > 0:
                    pred_target = i
            if i < len(path_as_strings):
                subset_path = True

    return pred_target, subset_path

In [74]:
path_df

,new_col
0,3C0
1,1K0
2,3O0
3,"3Y0,1AB0"
4,3AF0
5,1AR0
6,"3AZ0,1BD0"
7,3BG0
8,"3BQ0,1BR0"
9,1BZ0


In [75]:
def score():
    count = []
    bleu_score = []
    j_coeff = []
    l_dist = []
    path_mismatch_count = []
    traverse_check_count = []
    order_mismatch_count = []
    subset_path_count = []
    for i in range(test_data.shape[0]):
        curr_feat = np.array([test_data.iloc[i, 0:X.shape[1]]])
        path, label = predict(curr_feat)
        actual_path = test_data.iloc[i, -1]

        actual_path_tok = [label_indices[char] for char in actual_path]
        pred_path_tok = [label_indices[char] for char in path]

        # j_coeff.append(super().get_j_coeff(actual_path_tok, pred_path_tok))

        print('actual vs predicted: ', test_data.iloc[i, -1], ' vs ', ' '.join(
            path), 'labels: ', y[i], label[0])
        count.append(y[i] == label[0])
        # print('Actual path -- ', actual_path)
        # print('Pred path -- ', path)
#         if actual_path != path:
#             print(' -- Path mismatch -- ')
#             if sorted(actual_path) == sorted(path):
#                 print(' -- Order mismatch -- ')
#                 order_mismatch_count.append(1)
#             else:
#                 path_mismatch_count.append(1)
#                 pred_target, subset_path = self.check_path(path)
#                 subset_path_count.append(subset_path)
#                 if pred_target != -1 and pred_target == self.df.iloc[i, self.X.shape[1]+1]:
#                     traverse_check_count.append(1)


        path = list(''.join(path))
        actual_path = list(''.join(test_data.iloc[i, -1]))
        bleu_score.append(sentence_bleu([actual_path], path))

#         lev_path = []
#         for i in range(len(path)):
#             if i in ['S','L','R','E']:
#                 lev_path.append(i)
#         l_dist.append(distance.levenshtein(
#             self.df.iloc[i, self.X.shape[1]].replace(' ', ''), ''.join(lev_path)))


    print('\nLabel accuracy - ', np.mean(count))
#     print('Path metric (Jaccard) - ', np.mean(j_coeff))
#     print('Path metric (Levenshtein) - ', np.mean(l_dist))
#     print('Path mismatch count - ', np.sum(path_mismatch_count))
#     print('Right traverse count - ', np.sum(traverse_check_count))
#     print('Order mismatch count - ', np.sum(order_mismatch_count))
#     print('Subset path count - ', np.sum(subset_path_count))
    print('Bleu score of paths - ', np.mean(bleu_score))

In [76]:
score()

actual vs predicted:  ['S', '3C0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1K0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3O0', 'E']  vs  S E labels:  0 0


/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it cont

actual vs predicted:  ['S', '3Y0', '1AB0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3AF0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1AR0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3AZ0', '1BD0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3BG0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3BQ0', '1BR0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1BZ0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3CG0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1CS0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3CY0', '1CZ0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1DN0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3DR0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '2EG1', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '4EL0', '1EP0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '2EX1', 'E']  vs  S E labels:  0 0
actual vs predic

actual vs predicted:  ['S', '4CIQ1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '1CJO1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3CJZ1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '1CKV1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '1CLO1', '1CLP1', 'E']  vs  S E labels:  2 1
actual vs predicted:  ['S', '1CLX1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3CMP1', 'E']  vs  S E labels:  2 1
actual vs predicted:  ['S', '1CNP1', '3CMZ1', '2CMX0', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3CNW1', '4CNU1', 'E']  vs  S E labels:  2 1
actual vs predicted:  ['S', '1COX1', '3COW1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '1CPW1', 'E']  vs  S E labels:  2 1
actual vs predicted:  ['S', '1CQU1', '3CQT1', 'E']  vs  S E labels:  2 1
actual vs predicted:  ['S', '1CRX1', '1CRZ1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3CSY1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3CUV1', 'E']

### Path invariance trials

In [46]:
## Import nnum, vnum, nodes, csplit, split_df,
## frame

splits = pd.read_csv('../../data/raw/splits.csv', delimiter=",", index_col=0)
csplit = pd.read_csv('../../data/raw/csplit.csv', delimiter=",")
frame = pd.read_csv('../../data/raw/frame.csv', delimiter=",",index_col=0)

# frame = frame.drop(["Unnamed: 0"], axis=1)
frame = frame.rename(columns={"var": "variable"})
# bin_labels = bin_labels.rename(columns={"Unnamed: 0": "label", "label_list": "bins"})

In [47]:
frame

,variable,n,wt,dev,yval,complexity,ncompete,nsurrogate,yval2.,yval2..1,yval2..2,yval2..3,yval2..4,yval2..5,yval2..6,yval2.nodeprob
1,Petal.Length,150,150,100,1,0.500,3,3,1.0,50.0,50.0,50.0,0.333333,0.333333,0.333333,1.000000
2,<leaf>,50,50,0,1,0.000,0,0,1.0,50.0,0.0,0.0,1.000000,0.000000,0.000000,0.333333
3,Petal.Width,100,100,50,2,0.440,3,3,2.0,0.0,50.0,50.0,0.000000,0.500000,0.500000,0.666667
6,Petal.Length,54,54,5,2,0.020,3,0,2.0,0.0,49.0,5.0,0.000000,0.907407,0.092593,0.360000
12,Petal.Width,48,48,1,2,0.010,3,0,2.0,0.0,47.0,1.0,0.000000,0.979167,0.020833,0.320000
24,<leaf>,47,47,0,2,0.000,0,0,2.0,0.0,47.0,0.0,0.000000,1.000000,0.000000,0.313333
25,<leaf>,1,1,0,3,0.000,0,0,3.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,0.006667
13,Petal.Width,6,6,2,3,0.010,3,2,3.0,0.0,2.0,4.0,0.000000,0.333333,0.666667,0.040000
26,Sepal.Length,3,3,1,2,0.010,3,0,2.0,0.0,2.0,1.0,0.000000,0.666667,0.333333,0.020000
52,<leaf>,2,2,0,2,0.000,0,0,2.0,0.0,2.0,0.0,0.000000,1.000000,0.000000,0.013333


In [59]:
## Generate nnum, vnum, nodes(split_df and csplit - 2L if necessary)

temp_frame = frame

nc = temp_frame[["ncompete", "nsurrogate"]]

index = np.cumsum((frame[["variable"]]!="<leaf>").values + nc[["ncompete"]].values + nc[["nsurrogate"]].values)

index_df = pd.DataFrame((np.insert(index,0,0)+1)[:-1], columns=["i"], index=frame.index)

temp_frame = pd.concat([temp_frame, index_df], axis=1)

# temp_frame[temp_frame[["var"]]=="<leaf>"]
# temp_frame.where(temp_frame[["var"]]=="<leaf>")
# temp_frame.loc[temp_frame[["variable"]]=="<leaf>", "index"] = 0
temp_frame.i[temp_frame.variable == "<leaf>"] = 0

/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [60]:
temp_frame

,variable,n,wt,dev,yval,complexity,ncompete,nsurrogate,yval2.,yval2..1,yval2..2,yval2..3,yval2..4,yval2..5,yval2..6,yval2.nodeprob,i
1,Petal.Length,150,150,100,1,0.500,3,3,1.0,50.0,50.0,50.0,0.333333,0.333333,0.333333,1.000000,1
2,<leaf>,50,50,0,1,0.000,0,0,1.0,50.0,0.0,0.0,1.000000,0.000000,0.000000,0.333333,0
3,Petal.Width,100,100,50,2,0.440,3,3,2.0,0.0,50.0,50.0,0.000000,0.500000,0.500000,0.666667,8
6,Petal.Length,54,54,5,2,0.020,3,0,2.0,0.0,49.0,5.0,0.000000,0.907407,0.092593,0.360000,15
12,Petal.Width,48,48,1,2,0.010,3,0,2.0,0.0,47.0,1.0,0.000000,0.979167,0.020833,0.320000,19
24,<leaf>,47,47,0,2,0.000,0,0,2.0,0.0,47.0,0.0,0.000000,1.000000,0.000000,0.313333,0
25,<leaf>,1,1,0,3,0.000,0,0,3.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,0.006667,0
13,Petal.Width,6,6,2,3,0.010,3,2,3.0,0.0,2.0,4.0,0.000000,0.333333,0.666667,0.040000,23
26,Sepal.Length,3,3,1,2,0.010,3,0,2.0,0.0,2.0,1.0,0.000000,0.666667,0.333333,0.020000,29
52,<leaf>,2,2,0,2,0.000,0,0,2.0,0.0,2.0,0.0,0.000000,1.000000,0.000000,0.013333,0


In [61]:
nodes = temp_frame[["n", "ncompete", "nsurrogate", "i"]]

In [62]:
nnum = list(temp_frame.index) # row names of temp_frame

In [63]:
feature_names = ["Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width"]

vnum = list(map(feature_names.index, splits.index))

In [64]:
nodes

,n,ncompete,nsurrogate,i
1,150,3,3,1
2,50,0,0,0
3,100,3,3,8
6,54,3,0,15
12,48,3,0,19
24,47,0,0,0
25,1,0,0,0
13,6,3,2,23
26,3,3,0,29
52,2,0,0,0


In [80]:
nnum

[1, 2, 3, 6, 12, 24, 25, 13, 26, 52, 53, 27, 7, 14, 28, 29, 15]

In [81]:
def return_yval(path): # [1,0,0,0]
    node = 0
    nspl = 1
    i = 0
    while nspl != 0:
        npos = nnum[node] # i)0, 
        nspl = nodes.iloc[npos-1][3] # i)1
        var = vnum[nspl]
        # ncat
        temp = splits.iloc[nspl][3]
        if nspl > 0:
            print("nspl succeeded")
            if path[i] == 0: # i)1
                direction = -1
                i+=1
            else:
                direction = 1
                i+=1

            if direction == -1:
                print(node)
                if node == 0:
                    node = 1
                node = 2 * node
            else:
                print(node)
                if node == 0:
                    node = 1
                node = 2 * node + 1
            if len(path) == 1:
                nspl = 0
        else:
            print('nspl failed')
            print("leaf node -- ", node)
            yval = temp_frame.iloc[node][4]
    return temp_frame.iloc[node][4]

In [77]:
nodes.iloc[0]

n             150
ncompete        3
nsurrogate      3
i               1
Name: 1, dtype: int64

In [86]:

path = [1,1,0,1]
a = [1,0,0,0]
b = [1,1,1]
c = [0]
d = [1,1,1,1,1,1,0]
e = [1,0,1,0,0]
return_yval(e)

nspl succeeded
0
nspl failed
leaf node --  3


2

In [30]:
temp_frame

,variable,n,wt,dev,yval,complexity,ncompete,nsurrogate,yval2.,yval2..1,yval2..2,yval2..3,yval2..4,yval2..5,yval2..6,yval2.nodeprob,i
0,Petal.Length,150,150,100,1,0.500,3,3,1.0,50.0,50.0,50.0,0.333333,0.333333,0.333333,1.000000,1
1,<leaf>,50,50,0,1,0.000,0,0,1.0,50.0,0.0,0.0,1.000000,0.000000,0.000000,0.333333,0
2,Petal.Width,100,100,50,2,0.440,3,3,2.0,0.0,50.0,50.0,0.000000,0.500000,0.500000,0.666667,8
3,Petal.Length,54,54,5,2,0.020,3,0,2.0,0.0,49.0,5.0,0.000000,0.907407,0.092593,0.360000,15
4,Petal.Width,48,48,1,2,0.010,3,0,2.0,0.0,47.0,1.0,0.000000,0.979167,0.020833,0.320000,19
5,<leaf>,47,47,0,2,0.000,0,0,2.0,0.0,47.0,0.0,0.000000,1.000000,0.000000,0.313333,0
6,<leaf>,1,1,0,3,0.000,0,0,3.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,0.006667,0
7,Petal.Width,6,6,2,3,0.010,3,2,3.0,0.0,2.0,4.0,0.000000,0.333333,0.666667,0.040000,23
8,Sepal.Length,3,3,1,2,0.010,3,0,2.0,0.0,2.0,1.0,0.000000,0.666667,0.333333,0.020000,29
9,<leaf>,2,2,0,2,0.000,0,0,2.0,0.0,2.0,0.0,0.000000,1.000000,0.000000,0.013333,0


In [33]:
splits

,count,ncat,improve,index,adj
Petal.Length,150,-1,50.000000,2.45,0.000000
Petal.Width,150,-1,50.000000,0.80,0.000000
Sepal.Length,150,-1,34.164050,5.45,0.000000
Sepal.Width,150,1,19.038508,3.35,0.000000
Petal.Width,0,-1,1.000000,0.80,1.000000
Sepal.Length,0,-1,0.920000,5.45,0.760000
Sepal.Width,0,1,0.833333,3.35,0.500000
Petal.Width,100,-1,38.969404,1.75,0.000000
Petal.Length,100,-1,37.353535,4.75,0.000000
Sepal.Length,100,-1,10.686869,6.15,0.000000


In [96]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')